In [4]:
import numpy
import pandas
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm

import tensorflow
from tensorflow import keras

from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

Using TensorFlow backend.


In [107]:
from gensim.models import word2vec

/opt/anaconda3/envs/gensim_env/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [89]:
# data preprocessor

data = open('./selfmade_label.txt').read()
labels, texts = [], []
for i, line in enumerate(data.split('\n')):
    # print(line)
    content = line.split(' ')
    # print(content)
    if len(content) >= 2:
        labels.append(content[0])
        texts.append(content[1])

trainDF = pandas.DataFrame()
trainDF['label'] = labels
trainDF['text'] = texts

print(trainDF)

         label  text
0   __label__1    生活
1   __label__2    程式
2   __label__2    電腦
3   __label__2    理化
4   __label__2    化學
5   __label__2    物理
6   __label__2    科技
7   __label__2    研究
8   __label__2    實驗
9   __label__2    科學
10  __label__2    工程
11  __label__2  地球科學
12  __label__2  材料科學
13  __label__2  化學工程
14  __label__2    電機
15  __label__1    商學
16  __label__1    管理
17  __label__1    財金
18  __label__1    企管
19  __label__1    藝術
20  __label__1    設計
21  __label__1    創作
22  __label__1    商業
23  __label__1    貿易
24  __label__1    經濟
25  __label__1    手作
26  __label__1    外語
27  __label__1    語言
28  __label__2    電子
29  __label__2    資訊
30  __label__2  自動控制
31  __label__3    醫學
32  __label__3    解剖
33  __label__3    藥物
34  __label__3    腫瘤
35  __label__3    開刀
36  __label__3    確診
37  __label__3   心理學
38  __label__3  公共衛生
39  __label__3   營養學
40  __label__3    神經
41  __label__3    知覺


In [90]:
train_x = trainDF['text']
train_y = trainDF['label']

# __label__n to n
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)

[0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 2 2 2 2 2 2
 2 2 2 2 2]


In [91]:
# load trained word2vec
# in the dict called embeddings_index: 'key':[vec1, vec2, vec3, ...]

embeddings_index = {}
for i, line in enumerate(open('word2vec_model.vec')):
    values = line.split()
    embeddings_index[values[0]] = numpy.asarray(values[1:], dtype='float32')

In [92]:
# building an elasticsearch
token = text.Tokenizer()
token.fit_on_texts(trainDF['text'])
word_index = token.word_index # a dictionary

# guaranteeing they all have the same length of vectors
train_seq_x = sequence.pad_sequences(token.texts_to_sequences(train_x), maxlen=10)
valid_seq_x = sequence.pad_sequences(token.texts_to_sequences(valid_x), maxlen=10)

# arranging all words in my model
embedding_matrix = numpy.zeros((len(word_index), 250))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    print(word, i)
    print(embedding_vector)
    if embedding_vector is not None:
        embedding_matrix[i-1] = embedding_vector

生活 1
[-2.04146004e+00  2.34759235e+00 -1.09917617e+00  2.49099553e-01
  7.47409821e-01  1.74174285e+00 -1.60426509e+00 -1.83213845e-01
  8.64049718e-02  1.49117517e+00  6.64404035e-01  1.31039500e+00
  1.71245062e+00 -1.66836846e+00  1.95928705e+00  3.53052974e-01
 -9.47477400e-01 -2.20317245e+00  2.61917301e-02  8.07726204e-01
 -6.18626297e-01  1.60817176e-01 -9.00186718e-01 -3.01009583e+00
 -5.18941045e-01  6.04956746e-01  1.41204730e-01  2.05467880e-01
  1.50135255e+00 -2.51268196e+00  1.96626440e-01  5.80509961e-01
 -1.31144476e+00  1.39519855e-01 -3.12311471e-01  1.15375149e+00
  1.52344331e-02 -3.76921225e+00  1.28778827e+00 -3.59516591e-01
  3.79259497e-01 -2.02474689e+00 -1.35465252e+00 -1.04026258e+00
 -1.14514083e-02 -2.36228615e-01 -2.75094748e-01  1.43739998e+00
 -1.76576638e+00 -1.77076530e+00  1.20055087e-01  2.33741117e+00
 -7.86156833e-01  1.46020710e+00 -7.00652599e-01 -5.62083364e-01
 -1.35504329e+00 -3.32448387e+00 -2.17608690e+00  7.62683213e-01
 -1.29334420e-01  2.

In [96]:
classifier = linear_model.LogisticRegression()
model = classifier.fit(embedding_matrix, train_y)

model = word2vec.Word2Vec.load("word2vec.model")

In [118]:
high_tfidfs = open('./high_tfidfs.txt').read()
text = []
for i, word in enumerate(high_tfidfs.split('\n')):
    text.append(word)

text_vector = []
valid_bit = []

for i in text:
    try:
        text_vector.append(model.wv.get_vector(i))
        valid_bit.append(1)
    except KeyError:
        text_vector.append('error')
        valid_bit.append(0)

In [122]:
valid_text_data = []

for i, j in enumerate(text):
    if valid_bit[i] == 1:
        kind = classifier.predict([text_vector[i]])
        if kind == [0]:
            print('the word', j, 'classified to', '0: 一類')
        elif kind == [1]:
            print('the word', j, 'classified to', '1: 二類')
        elif kind == [2]:
            print('the word', j, 'classified to', '2: 三類')


the word 資料 classified to 1: 二類
the word 科學 classified to 1: 二類
the word 專頁 classified to 0: 一類
the word 粉絲 classified to 0: 一類
the word 組織 classified to 0: 一類
the word 工程 classified to 1: 二類
the word 部落 classified to 0: 一類
the word 學習 classified to 0: 一類
the word 科技 classified to 1: 二類
the word 教育 classified to 0: 一類
the word 查看 classified to 1: 二類
the word 資訊 classified to 1: 二類
the word 協助 classified to 0: 一類
the word 瞭解 classified to 2: 三類
the word 內容 classified to 0: 一類
the word 採取 classified to 0: 一類
the word 動作 classified to 0: 一類
the word 資料 classified to 1: 二類
the word 職涯 classified to 0: 一類
the word 科學家 classified to 1: 二類
the word 活動 classified to 0: 一類
the word 數據 classified to 1: 二類
the word 加值 classified to 0: 一類
the word 計畫 classified to 1: 二類
the word 工作 classified to 0: 一類
the word 上午 classified to 2: 三類
the word 這跟 classified to 0: 一類
the word 師和 classified to 0: 一類
the word 幹話 classified to 0: 一類
the word 企劃 classified to 0: 一類
the word 講座 classified to 0: 一類
the wor

the word 科學 classified to 1: 二類
the word 分析 classified to 1: 二類
the word 文章 classified to 0: 一類
the word 產業 classified to 0: 一類
the word 一個 classified to 0: 一類
the word 準備 classified to 0: 一類
the word 廣告主 classified to 0: 一類
the word 數位 classified to 1: 二類
the word 之前 classified to 0: 一類
the word 流程 classified to 0: 一類
the word 有趣 classified to 0: 一類
the word 資料 classified to 1: 二類
the word 科學 classified to 1: 二類
the word 專頁 classified to 0: 一類
the word 粉絲 classified to 0: 一類
the word 組織 classified to 0: 一類
the word 工程 classified to 1: 二類
the word 部落 classified to 0: 一類
the word 學習 classified to 0: 一類
the word 科技 classified to 1: 二類
the word 教育 classified to 0: 一類
the word 查看 classified to 1: 二類
the word 資訊 classified to 1: 二類
the word 協助 classified to 0: 一類
the word 瞭解 classified to 2: 三類
the word 內容 classified to 0: 一類
the word 採取 classified to 0: 一類
the word 動作 classified to 0: 一類
the word 資料 classified to 1: 二類
the word 職涯 classified to 0: 一類
the word 科學家 classified to 1: 二類
the wo

the word 執行 classified to 0: 一類
the word 知識 classified to 1: 二類
the word 點過 classified to 0: 一類
the word 分析 classified to 1: 二類
the word 面向 classified to 0: 一類
the word 關鍵 classified to 0: 一類
the word 步驟 classified to 1: 二類
the word 應用 classified to 1: 二類
the word 職涯 classified to 0: 一類
the word 新創 classified to 0: 一類
the word 改變 classified to 2: 三類
the word 公司 classified to 0: 一類
the word 職場 classified to 0: 一類
the word 建議 classified to 2: 三類
the word 領航者 classified to 0: 一類
the word 面試 classified to 0: 一類
the word 觀看 classified to 0: 一類
the word 環境 classified to 1: 二類
the word 培養 classified to 0: 一類
the word 播放 classified to 0: 一類
the word 快速 classified to 1: 二類
the word 有個 classified to 0: 一類
the word 參考 classified to 0: 一類
the word 活動 classified to 0: 一類
the word 華碩 classified to 1: 二類
the word 台北 classified to 0: 一類
the word 資料 classified to 1: 二類
the word 科技 classified to 1: 二類
the word 領導 classified to 0: 一類
the word 傳遞 classified to 1: 二類
the word 倫比 classified to 0: 一類
the wor

the word 科學 classified to 1: 二類
the word 經驗 classified to 0: 一類
the word 相關 classified to 0: 一類
the word 知識 classified to 1: 二類
the word 同學 classified to 0: 一類
the word 分析 classified to 1: 二類
the word 建議 classified to 2: 三類
the word 未來 classified to 0: 一類
the word 課程 classified to 0: 一類
the word 累積 classified to 0: 一類
the word 數據 classified to 1: 二類
the word 創造 classified to 0: 一類
the word 優勢 classified to 0: 一類
the word 本科 classified to 0: 一類
the word 咖啡 classified to 0: 一類
the word 思考 classified to 0: 一類
the word 資料 classified to 1: 二類
the word 媒體 classified to 0: 一類
the word 讀者 classified to 0: 一類
the word 廣告 classified to 0: 一類
the word 專案 classified to 1: 二類
the word 內容 classified to 0: 一類
the word 分群 classified to 0: 一類
the word 數據 classified to 1: 二類
the word 科學 classified to 1: 二類
the word 執行 classified to 0: 一類
the word 知識 classified to 1: 二類
the word 點過 classified to 0: 一類
the word 分析 classified to 1: 二類
the word 面向 classified to 0: 一類
the word 關鍵 classified to 0: 一類
the word